<a href="https://colab.research.google.com/github/bachvu98/Policy-NLP/blob/master/Loading_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introduction
This file loads and persists data from the OPP-115 corpus

# Import required dependencies

In [1]:
from os import listdir
import pandas as pd
import numpy as np
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import *
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from lxml import etree, html
import json
import time

# Read in sites table
Grab the sites table

In [2]:
sites = pd.read_csv('https://raw.githubusercontent.com/bachvu98/Policy-NLP/master/OPP-115_v1_0/OPP-115/documentation/websites_covered_opp115.csv',index_col=3,parse_dates=[4])
sites.head()

,Site UID,Site URL,Site Human-Readable Name,Site Check Date,In 115 Set?,Comments,Sectoral Data,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,...,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106
Policy UID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20,1,theatlantic.com,The Atlantic,2016-02-08,Yes,"Alexa Rank: 975 (Global), 289 (US)","Arts: Literature: Authors: A: Alcott, Louisa M...","Arts: Literature: Authors: M: Merwin, W. S.: W...",Arts: Literature: World Literature: American: ...,"Arts: Music: Composition: Composers: T: Toch, ...","Arts: Literature: Authors: H: Hemingway, Ernes...","Arts: Literature: Authors: P: Pinsky, Robert",Regional: North America: United States: Govern...,Regional: North America: United States: Govern...,Science: Social Sciences: Economics: People: S...,Society: Crime: Theft: Art and Antiquities: Wo...,Society: History: By Region: Middle East: Kurd...,Society: History: By Time Period: Nineteenth C...,Society: Issues: Crime and Justice: Prisons: A...,Arts: Architecture: History: Architects: O: Ol...,"Arts: Art History: Artists: P: Picasso, Pablo","Arts: Art History: Artists: S: Schiele, Egon","Arts: Literature: Authors: A: Alexie, Sherman","Arts: Literature: Authors: A: Amis, Martin","Arts: Literature: Authors: A: Auden, W. H.: Works","Arts: Literature: Authors: B: Barthelme, Frede...","Arts: Literature: Authors: B: Bass, Rick: Works","Arts: Literature: Authors: B: Bidart, Frank","Arts: Literature: Authors: B: Bishop, Elizabeth","Arts: Literature: Authors: B: Bogan, Louise","Arts: Literature: Authors: C: Carver, Raymond:...","Arts: Literature: Authors: D: Doty, Mark","Arts: Literature: Authors: E: Erdrich, Louise","Arts: Literature: Authors: F: Frost, Robert","Arts: Literature: Authors: G: Gallagher, Tess","Arts: Literature: Authors: G: Gioia, Dana","Arts: Literature: Authors: J: James, Henry","Arts: Literature: Authors: K: Kerouac, Jack","Arts: Literature: Authors: L: Levine, Philip","Arts: Literature: Authors: L: Lightman, Alan",...,Arts: Music: Styles: J: Jazz: Bands and Artist...,Arts: Music: Styles: J: Jazz: Bands and Artist...,Arts: Music: Styles: J: Jazz: History,Arts: Music: Styles: J: Jazz: News and Media: ...,Arts: Online Writing: Fiction: Short Stories,Arts: Performing Arts: Acting: Actors and Actr...,Arts: Television: Programs: Children's: Sesame...,Arts: Writers Resources: Style Guides,Computers: Artificial Intelligence: Philosophy,Computers: E-Books: News and Media,"Computers: Internet: History: People: Barlow, ...",Games: Board Games: Abstract: Battle Games: Ch...,Games: Video Games: Adventure: Graphical Adven...,Games: Video Games: Adventure: Graphical Adven...,Health: Specific Substances: Tobacco: History,News: Analysis and Opinion: Magazines and E-zines,News: Current Events: Business and Economy: An...,News: Magazines and E-zines,Recreation: Food: Drink: Beer: Reviews,Recreation: Outdoors: Hunting: Foxhunting: Art...,Reference: Education: Educational Testing: Opp...,Regional: Africa: Rwanda: Society and Culture:...,Regional: Africa: South Africa: Society and Cu...,Regional: Europe: United Kingdom: Society and ...,Regional: Europe: United Kingdom: Society and ...,Regional: Middl

Change interpretation of 115 set membership column

In [3]:
sites['In 115 Set?']=sites['In 115 Set?'].apply(lambda yn: True if yn == 'Yes' else False)
sites = sites.loc[sites['In 115 Set?'] == True]

Reinterpret the categories for the sites. Find the primary category and take the mode across all columns

In [4]:
sectors = sites[sites.columns[6:]]
sectors = sectors.applymap(lambda s: str(s).split(':')[0])
sectors.head()

,Sectoral Data,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,...,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106
Policy UID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20,Arts,Arts,Arts,Arts,Arts,Arts,Regional,Regional,Science,Society,Society,Society,Society,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,...,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Computers,Computers,Computers,Games,Games,Games,Health,News,News,News,Recreation,Recreation,Reference,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Science,Science,Science,Science,Science,Science
21,Kids and Teens,Regional,Arts,Arts,Arts,Arts,Arts,Arts,Regional,Sports,Arts,Arts,Arts,Arts,Arts,Arts,Regional,Regional,Regional,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Regional,Regional,Kids and Teens,Kids and Teens,Kids and Teens,...,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts
26,Society,Science,Society,Computers,Arts,Arts,Arts,Society,Regional,Society,Arts,Arts,Arts,Society,Society,Society,Arts,News,Science,Society,Society,Society,Arts,Arts,Arts,Arts,Arts,Computers,Health,Reference,Reference,Regional,Regional,Society,Society,Society,Sports,Arts,Arts,Arts,...,Computers,Home,News,News,News,Reference,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Regional,Science,Science,Science,Society,Society,Society,Society,Society,Society,Society,Society,Society
32,Home,Home,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
33,Kids and Teens,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Arts,Business,Recreation,Regional,Regional,World,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [5]:
s = []
for i in range(0,sectors.shape[0]):
    
    sec = sectors.iloc[i][sectors.iloc[i]!='nan'].mode()
    sec = sec.iloc[0] if len(sec) > 0 else 'None'
    s.append(sec)

Drop and append

In [30]:
sites = sites.drop(sites.columns[6:], axis=1)
sites['Sector'] = s
sites.head()

,Site UID,Site URL,Site Human-Readable Name,Site Check Date,In 115 Set?,Comments,Sector
Policy UID,,,,,,,
20,1,theatlantic.com,The Atlantic,2016-02-08,True,"Alexa Rank: 975 (Global), 289 (US)",Arts
21,2,imdb.com,IMDb,2016-02-08,True,"Alexa Rank: 49 (Global), 27 (US)",Arts
26,3,nytimes.com,New York Times,2016-02-08,True,"Alexa Rank: 101 (Global), 22 (US)",Arts
32,4,theverge.com,The Verge,2016-02-08,True,"Alexa Rank: 525 (Global), 230 (US)",Home
33,5,nbc.com,NBC Universal,2016-02-08,True,"Alexa Rank: 1548 (Global), 426 (US)",Arts


In [8]:
sites.to_csv("/content/drive/My Drive/OPP-115/OPP-115/sites.csv")

# Read in policies table
Grab the policies table

In [32]:
policies = pd.read_csv('https://raw.githubusercontent.com/bachvu98/Policy-NLP/master/OPP-115_v1_0/OPP-115/documentation/policies_opp115.csv',index_col=0,parse_dates=[2,3])
policies = policies.drop('Unnamed: 4',axis=1)
policies.head()

,Policy URL,Policy collection date,Policy last updated date
Policy UID,,,
20,theatlantic.com/privacy-policy/,2015-07-02,2015-01-01
21,imdb.com/privacy,2015-07-02,2014-12-05
26,nytimes.com/privacy,2015-07-08,2015-06-10
32,voxmedia.com/privacy-policy,2015-07-02,2014-05-01
33,nbcuniversal.com/privacy/full-privacy-policy,2015-07-02,2015-01-14


Get sanitized policies from HTML files

In [33]:
base_dir = '/content/drive/My Drive/OPP-115/OPP-115/sanitized_policies/'
files = [f for f in listdir(base_dir) if f.endswith('.html')]

In [34]:
#Check matching policy ids
pids = sites.index.tolist()
for f in files:
    pid = np.int(f.split('_')[0])
    if pid not in pids:
        print('MISSING '+str(pid))

Combine policies from different HTML files


In [35]:
txt = []
pids = []
for f in files:
    with open(base_dir + f, 'r') as pg:
        page = pg.read()
        txt.append(page)
        pid = np.int(f.split('_')[0])
        pids.append(pid)

Create dataframe of texts and join it to policies frame


In [36]:
txts = pd.DataFrame({'policy_text': txt},index=pids)
txts.index.name = 'Policy UID'
policies = pd.merge(policies,txts,left_index=True,right_index=True,how='outer')
policies.head()

,Policy URL,Policy collection date,Policy last updated date,policy_text
Policy UID,,,,
20,theatlantic.com/privacy-policy/,2015-07-02,2015-01-01,<strong> Privacy Policy </strong> <br> <br> <s...
21,imdb.com/privacy,2015-07-02,2014-12-05,"IMDb Privacy Notice <br> <br>|||Last Updated, ..."
26,nytimes.com/privacy,2015-07-08,2015-06-10,<strong> Privacy Policy </strong> <br> <br> La...
32,voxmedia.com/privacy-policy,2015-07-02,2014-05-01,Vox Media Privacy Policy <br> <br>|||<strong> ...
33,nbcuniversal.com/privacy/full-privacy-policy,2015-07-02,2015-01-14,Full Privacy Policy <br> <br> Last updated: 14...


For each policy, create a dataframe of segments with unique segment index


In [37]:
dfs = []
for i in range(0,policies.shape[0]):
    tmp = pd.DataFrame(policies['policy_text'].iloc[i].split('|||'),columns=['segments'])
    tmp['Policy UID'] = policies.index.values[i]
    tmp.index.name = 'segment_id'
    dfs.append(tmp)

In [60]:
segments = pd.concat(dfs,axis=0).reset_index().set_index(['Policy UID','segment_id'])
segments.head()

segments
Policy UID segment_id                                                   
20         0           <strong> Privacy Policy </strong> <br> <br> <s...
           1           This privacy policy does not apply to Sites ma...
           2           By visiting our Sites, you are accepting the p...
           3           <strong> What Information Is Collected? </stro...
           4           <strong> Personally Identifiable Information <...

# Read in annotations table
Combine annotations from CSV files

In [39]:
url = "/content/drive/My Drive/OPP-115/OPP-115/annotations/"
files = [f for f in listdir(url) if f.endswith('.csv')]

In [40]:
names = """annotation_id 
        batch_id 
        annotator_id 
        policy_id 
        segment_id 
        category_name 
        attributes_value_pairs 
        date 
        policy_url""".split()

types = {'annotation_id': np.int,
        'batch_id': str,
        'annotator_id': np.int,
        'policy_id': np.int,
        'segment_id': np.int,
        'category_name': str}

ann_list = []
pids = []
for f in files:
    df = pd.read_csv(url + f,header=None,names=names,na_values={'date': 'Not specified'},
                 parse_dates=[7],index_col=0)
    ann_list.append(df)
    pids.append(np.int(f.split('_')[0]))

In [41]:
annotations = pd.concat(ann_list,axis=0,keys=pids,names=['Policy UID','annotation_id'])
annotations = annotations.drop('policy_id',axis=1)
annotations.head()

batch_id  ...                                         policy_url
Policy UID annotation_id                                                  ...                                                   
1539       15594          test_category_labeling_highlight_fordham_bbbbb  ...  https://www.geocaching.com/about/privacypolicy...
           15595          test_category_labeling_highlight_fordham_bbbbb  ...  https://www.geocaching.com/about/privacypolicy...
           15596          test_category_labeling_highlight_fordham_bbbbb  ...  https://www.geocaching.com/about/privacypolicy...
           15597          test_category_labeling_highlight_fordham_bbbbb  ...  https://www.geocaching.com/about/privacypolicy...
           15598          test_category_labeling_highlight_fordham_bbbbb  ...  https://www.geocaching.com/about/privacypolicy...

[5 rows x 7 columns]

In [ ]:
attr_values = pd.DataFrame(data=None,columns=['annotation_id','start_idx','end_idx','attribute','text','value'])
template = dict.fromkeys('startIndexInSegment endIndexInSegment selectedText value'.split())

In [ ]:
#BATCH PROCESSING OF ATTRIBUTE-VALUE PAIRS: THIS NEEDS TO BE DONE MANUALLY!
attr_values1 = pd.DataFrame(data=None,columns=['annotation_id','start_idx','end_idx','attribute','text','value'])
t0 = time.time()
print('Starting at ' + str(t0))
for i in range(0,annotations.shape[0]):    #Note manual adjustment of batch size here...
    
    attr_val = annotations['attributes_value_pairs'].iloc[i]
    ann_id = annotations.index.values[i][-1]
    
    obj = json.loads(attr_val)
    keys = list(obj.keys())
    
    for k in keys:
        obj2 = dict(template, **obj[k]) #Ensures at a minimum we get the empty template data.
        df = pd.DataFrame({'annotation_id': [ann_id],
                           'start_idx': [obj2['startIndexInSegment']],
                           'end_idx': [obj2['endIndexInSegment']],
                           'attribute': [k],
                           'text': [obj2['selectedText']],
                           'value': [obj2['value']]})
        attr_values1 = attr_values1.append(df,ignore_index=True)
t1 = time.time()
print('Finished at ' + str(t1))
print('Total elapsed time ' + str(t1-t0))
attr_values = attr_values.append(attr_values1,ignore_index=True)
attr_values.info()

In [ ]:
#Set up indices
attr_values.index.name = 'text_id'
attr_values = attr_values.set_index(['annotation_id'], append=True).reorder_levels(['annotation_id','text_id'])

In [ ]:
annotations.to_csv("/content/drive/My Drive/OPP-115/OPP-115/annotations.csv")
attr_values.to_csv("/content/drive/My Drive/OPP-115/OPP-115/text_selected.csv")

# Merge policy segments to corresponding categories


In [61]:
joined = pd.merge(annotations,segments,on=['Policy UID','segment_id'],how='outer')
joined['category_name'] = joined['category_name'].fillna(value='None')
joined = joined.drop(['batch_id','attributes_value_pairs','date','annotator_id','policy_url'],axis=1)
joined.head()

,segment_id,category_name,segments
Policy UID,,,
1539,0,Policy Change,<strong> PRIVACY POLICY </strong> <br> <br> La...
1539,0,Other,<strong> PRIVACY POLICY </strong> <br> <br> La...
1539,0,Other,<strong> PRIVACY POLICY </strong> <br> <br> La...
1539,1,Data Security,"Groundspeak, Inc. is dedicated to protecting y..."
1539,1,Other,"Groundspeak, Inc. is dedicated to protecting y..."


There are usually cases where a single segment belong to multiple categories.

In [62]:
joined.groupby(['Policy UID','segment_id']).agg(lambda x: x.nunique())['category_name']

Policy UID  segment_id
20          0             1
            1             1
            2             2
            3             1
            4             2
                         ..
1713        84            3
            85            1
            86            1
            87            1
            88            1
Name: category_name, Length: 3792, dtype: int64

In this case, we select the category name that appears most often in each segment.

In [63]:
#Get the mode of each segment
mode_categories = joined.groupby(['Policy UID','segment_id']).agg(lambda x: x.value_counts().index[0])
mode_categories = mode_categories.reset_index()
mode_categories.head()

,Policy UID,segment_id,category_name,segments
0,20,0,Other,<strong> Privacy Policy </strong> <br> <br> <s...
1,20,1,Other,This privacy policy does not apply to Sites ma...
2,20,2,Policy Change,"By visiting our Sites, you are accepting the p..."
3,20,3,First Party Collection/Use,<strong> What Information Is Collected? </stro...
4,20,4,First Party Collection/Use,<strong> Personally Identifiable Information <...


Persist data to CSV file

In [66]:
mode_categories.to_csv("/content/drive/My Drive/OPP-115/OPP-115/segment_categories.csv")